# Running a QE calculation with AiiDA


In [2]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: fair-workflows.

In [3]:
from aiida.engine import run
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import bulk

In [4]:
%%capture
# Install aiida-pseudo
!pip install aiida-pseudo

In [5]:
%%capture
# Install PPs using aiida-pseudo
!aiida-pseudo install sssp -x PBEsol -p efficiency

In [6]:
# Load the code configured for ``pw.x``. Make sure to replace this string
# with the label of a ``Code`` that you configured in your profile.
code = load_code('qe-7.3-pw@localhost')

builder = code.get_builder()

from aiida_quantumespresso.workflows.pw.bands import PwBandsWorkChain

# builder = PwBandsWorkChain.get_builder_from_protocol(code=code, structure=structure)
# builder

In [7]:
# Create a silicon fcc crystal
structure = StructureData(ase=bulk('Si', 'fcc', 5.43))
builder.structure = structure

In [8]:
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
builder.pseudos = pseudo_family.get_pseudos(structure=structure)

In [10]:
# Request the recommended wavefunction and charge density cutoffs
# for the given structure and energy units.
cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

parameters = Dict({
    'CONTROL': {
        'calculation': 'scf'
    },
    'SYSTEM': {
        'ecutwfc': cutoff_wfc,
        'ecutrho': cutoff_rho,
    }
})
builder.parameters = parameters

In [11]:
# Generate a 2x2x2 Monkhorst-Pack mesh
kpoints = KpointsData()
kpoints.set_kpoints_mesh([2, 2, 2])
builder.kpoints = kpoints

In [12]:
# Run the calculation on 1 CPU and kill it if it runs longer than 1800 seconds.
# Set ``withmpi`` to ``False`` if ``pw.x`` was compiled without MPI support.
builder.metadata.options = {
    'resources': {
        'num_machines': 1,
    },
    'max_wallclock_seconds': 1800,
    'withmpi': False,
}

In [13]:
results, node = run.get_node(builder)
print(f'Calculation: {node.process_class}<{node.pk}> {node.process_state.value} [{node.exit_status}]')
print(f'Results: {results}')
assert node.is_finished_ok, f'{node} failed: [{node.exit_status}] {node.exit_message}'

Calculation: <class 'aiida_quantumespresso.calculations.pw.PwCalculation'><315> finished [0]
Results: {'remote_folder': <RemoteData: uuid: 01ec192c-b083-4e33-a47a-ce8e4f838f72 (pk: 316)>, 'retrieved': <FolderData: uuid: c5f00cbc-8901-40fd-92b1-595b7ce57807 (pk: 317)>, 'output_band': <BandsData: uuid: cbbbff37-c2d5-4072-8036-6fc3e1800a44 (pk: 318)>, 'output_trajectory': <TrajectoryData: uuid: 66a131a5-2571-477f-a34e-4ef5dd4dd8ee (pk: 319)>, 'output_parameters': <Dict: uuid: 0e82b873-39c4-4b71-b456-98e654ecafd5 (pk: 320)>}
